In [0]:
!pip install -q h5py pyyaml

In [2]:
# We'll use the MNIST dataset to train our model to demonstrate saving weights. 
# To speed up these demonstration runs, only use the first 1000 examples:

from __future__ import absolute_import, division, print_function, unicode_literals

import os

!pip install -q tensorflow==2.0.0-alpha0
import tensorflow as tf
from tensorflow import keras

tf.__version__

     |████████████████████████████████| 79.9MB 2.0MB/s 
     |████████████████████████████████| 419kB 42.3MB/s 
     |████████████████████████████████| 61kB 22.0MB/s 
     |████████████████████████████████| 3.0MB 35.8MB/s 


'2.0.0-alpha0'

In [3]:
(train_images, train_labels), (test_images, test_labels) = \
    tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28*28) / 255.0
test_images = test_images[:1000].reshape(-1, 28*28) / 255.0

11493376/11490434 [==============================] - 0s 0us/step


In [0]:
def create_model():
    model = tf.keras.models.Sequential([
        keras.layers.Dense(512, activation='relu', input_shape=(784,)),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(10, activation='softmax')
    ])
    
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [8]:
model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [0]:
## Save checkpoints during training
# The primary use case is to automatically save checkpoints during and at the end of training.
checkpoint_path = 'training_1/cp.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create checkpoint callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

In [11]:
model.fit(train_images,
          train_labels,
          epochs=10,
          validation_data=(test_images, test_labels),
          callbacks=[cp_callback])

Train on 1000 samples, validate on 1000 samples
Epoch 1/10
 736/1000 [=====================>........] - ETA: 0s - loss: 1.3232 - accuracy: 0.6196
Epoch 00001: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 497us/sample - loss: 1.1645 - accuracy: 0.6610 - val_loss: 0.6941 - val_accuracy: 0.7920
Epoch 2/10
 736/1000 [=====================>........] - ETA: 0s - loss: 0.4313 - accuracy: 0.8845
Epoch 00002: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 253us/sample - loss: 0.4213 - accuracy: 0.8830 - val_loss: 0.5338 - val_accuracy: 0.8390
Epoch 3/10
 704/1000 [====================>.........] - ETA: 0s - loss: 0.2891 - accuracy: 0.9318
Epoch 00003: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 245us/sample - loss: 0.2778 - accuracy: 0.9320 - val_loss: 0.4580 - val_accuracy: 0.8670
Epoch 4/10
 736/1000 [=====================>........] - ETA: 0s - loss: 0.2003 - accuracy: 0.9565
Epoch

In [12]:
!ls {checkpoint_dir}

checkpoint  cp.ckpt.data-00000-of-00001  cp.ckpt.index


In [13]:
# Create a new, untrained model.
# When restoring a model from only weights, you must have a model with the same architecture as the original model.
# Since it's the same model architecture, we can share weights despite that it's a different instance of the model.

# Now rebuild a fresh, untrained model, and evaluate it on the test set. An untrained model will perform at chance levels (~10% accuracy):
model = create_model()

loss, acc = model.evaluate(test_images, test_labels)

print('Untrained model, accuracy: {:5.2f}%'.format(100*acc))

1000/1000 [==============================] - 0s 111us/sample - loss: 2.3531 - accuracy: 0.0900
Untrained model, accuracy:  9.00%


In [14]:
# Then load the weights from the checkpoint, and re-evaluate:

model.load_weights(checkpoint_path)

loss, acc = model.evaluate(test_images, test_labels)
print('Untrained model, accuracy: {:5.2f}%'.format(100*acc))

1000/1000 [==============================] - 0s 73us/sample - loss: 0.4097 - accuracy: 0.8730
Untrained model, accuracy: 87.30%


In [0]:
## Checkpoint callback options
# The callback provides several options to give the resulting checkpoints unique names, and adjust the checkpointing frequency.
# Train a new model, and save uniquely named checkpoints once every 5-epochs:

# Include the epoch in the file name. (uses `str.fotmat`)
checkpoint_path = 'training_2/cp-{epoch:04d}.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = \
        tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                           verbose=1,
                                           save_weights_only=True,
                                           # Save weights, every 5-epochs
                                           period=5)

In [21]:
model = create_model()
model.save_weights(checkpoint_path.format(epoch=0))
model.fit(train_images,
          train_labels,
          epochs=50,
          callbacks=[cp_callback],
          validation_data=(test_images, test_labels),
          verbose=0)


Epoch 00005: saving model to training_2/cp-0005.ckpt

Epoch 00010: saving model to training_2/cp-0010.ckpt

Epoch 00015: saving model to training_2/cp-0015.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00025: saving model to training_2/cp-0025.ckpt

Epoch 00030: saving model to training_2/cp-0030.ckpt

Epoch 00035: saving model to training_2/cp-0035.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00045: saving model to training_2/cp-0045.ckpt

Epoch 00050: saving model to training_2/cp-0050.ckpt


In [22]:
!ls {checkpoint_dir}

checkpoint			  cp-0025.ckpt.index
cp-0000.ckpt.data-00000-of-00001  cp-0030.ckpt.data-00000-of-00001
cp-0000.ckpt.index		  cp-0030.ckpt.index
cp-0005.ckpt.data-00000-of-00001  cp-0035.ckpt.data-00000-of-00001
cp-0005.ckpt.index		  cp-0035.ckpt.index
cp-0010.ckpt.data-00000-of-00001  cp-0040.ckpt.data-00000-of-00001
cp-0010.ckpt.index		  cp-0040.ckpt.index
cp-0015.ckpt.data-00000-of-00001  cp-0045.ckpt.data-00000-of-00001
cp-0015.ckpt.index		  cp-0045.ckpt.index
cp-0020.ckpt.data-00000-of-00001  cp-0050.ckpt.data-00000-of-00001
cp-0020.ckpt.index		  cp-0050.ckpt.index
cp-0025.ckpt.data-00000-of-00001


In [23]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0050.ckpt'

In [25]:
model = create_model()
model.load_weights(latest)
loss, acc = model.evaluate(test_images, test_labels)
print('Restored model, accurcay: {:5.2f}'.format(100*acc))

1000/1000 [==============================] - 0s 111us/sample - loss: 0.4814 - accuracy: 0.8770
Restored model, accurcay: 87.70


In [26]:
## Manually save weights

# Save the weights
model.save_weights('./checkpoints/my_checkpoint')

# Restore the weight
model = create_model()
model.load_weights('./checkpoints/my_checkpoint')

loss,acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 115us/sample - loss: 0.4814 - accuracy: 0.8770
Restored model, accuracy: 87.70%


In [27]:
# The model and optimizer can be saved to a file that contains both their state (weights and variables), and the model configuration.
# This allows you to export a model so it can be used without access to the original python code.
# Since the optimizer-state is recovered you can even resume training from exactly where you left off
# Saving a fully-functional model is very useful—you can load them in TensorFlow.js 
# and then train and run them in web browsers, or convert them to run on mobile devices using TensorFlow Lite

# Keras provides a basic save format using the HDF5 standard. 

model = create_model()

model.fit(train_images, train_labels, epochs=5)

# Save entire model to a HDF5 file
model.save('my_model.h5')

Epoch 1/5
1000/1000 [==============================] - 0s 246us/sample - loss: 1.1391 - accuracy: 0.6830
Epoch 2/5
1000/1000 [==============================] - 0s 156us/sample - loss: 0.4258 - accuracy: 0.8790
Epoch 3/5
1000/1000 [==============================] - 0s 150us/sample - loss: 0.2789 - accuracy: 0.9240
Epoch 4/5
1000/1000 [==============================] - 0s 147us/sample - loss: 0.2087 - accuracy: 0.9500
Epoch 5/5
1000/1000 [==============================] - 0s 157us/sample - loss: 0.1568 - accuracy: 0.9630


In [29]:
# Recreate the exact same model, including weights and optimizer.
new_model = keras.models.load_model('my_model.h5')
new_model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_13 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_6 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_14 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [30]:
loss, acc = new_model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 104us/sample - loss: 0.4183 - accuracy: 0.8580
Restored model, accuracy: 85.80%


In [31]:
# This technique saves everything:

# The weight values
# The model's configuration(architecture)
# The optimizer configuration

# saved_model
model = create_model()
model.fit(train_images, train_labels, epochs=5)

Epoch 1/5
1000/1000 [==============================] - 0s 243us/sample - loss: 1.1501 - accuracy: 0.6730
Epoch 2/5
1000/1000 [==============================] - 0s 156us/sample - loss: 0.4092 - accuracy: 0.8820
Epoch 3/5
1000/1000 [==============================] - 0s 142us/sample - loss: 0.2942 - accuracy: 0.9250
Epoch 4/5
1000/1000 [==============================] - 0s 143us/sample - loss: 0.2112 - accuracy: 0.9550
Epoch 5/5
1000/1000 [==============================] - 0s 146us/sample - loss: 0.1460 - accuracy: 0.9680


In [33]:
# Create a saved_model, and place it in a time-stamped directory:

import time
saved_model_path = './saved_models/{}'.format(int(time.time()))

tf.keras.experimental.export_saved_model(model, saved_model_path)
saved_model_path

W0519 13:12:22.392745 139987019732864 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/saved_model/signature_def_utils_impl.py:253: build_tensor_info (from tensorflow.python.saved_model.utils_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
W0519 13:12:22.395452 139987019732864 tf_logging.py:161] Export includes no default signature!
W0519 13:12:22.928089 139987019732864 tf_logging.py:161] Export includes no default signature!


'./saved_models/1558271541'

In [34]:
!ls saved_models/

1558271541


In [35]:
# Reload a fresh keras model from the saved model.
new_model = tf.keras.experimental.load_from_saved_model(saved_model_path)
new_model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_15 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_7 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_16 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [36]:
model.predict(test_images).shape

(1000, 10)

In [37]:
# The model has to be compiled before evaluating.
# This step is not required if the saved model is only being deployed.

new_model.compile(optimizer=model.optimizer,  # keep the optimizer that was loaded
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Evaluate the restored model.
loss, acc = new_model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 112us/sample - loss: 0.4143 - accuracy: 0.8670
Restored model, accuracy: 86.70%
